# **MECS6616 Spring 2024 - Project 4**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/197115/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

**FOR PROJECT 4!!!**
- Apart from the link to your notebook, you are also required to submit the collected data file `data.pkl` and your chosen model checkpoint `dynamic.pth` to Coursework. You should put the link to your notebook in the "Comment" section of your submission.
- You can find a walkthrough of this project on Canvas under "Video Library" (03/26 Lecture), where we discuss key implementation details.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project4' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp24_project4.git

Cloning into 'mecs6616_sp24_project4'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 9.00 KiB | 1.50 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp24_project4/* /content/

In [ ]:
# Using ray for data collection is optional
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 8.8 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
import numpy as np
import random
np.random.seed(0)

class MPC:
    def __init__(self):
        np.random.seed(0)

        self.control_horizon = 5

        self.num_samples = 500

    def compute_action(self, dynamics, state, goal, action):
        best_action = np.zeros((dynamics.get_action_dim(), 1))
        min_cost = float('inf')


        initial_pos_ee = dynamics.compute_fk(state)

        initial_distance = np.linalg.norm(goal - initial_pos_ee)


        action_range = 4

        for _ in range(self.num_samples):
            sample_action = np.random.uniform(-action_range, action_range, (dynamics.get_action_dim(), 1))

            cumulative_state = np.copy(state)
            for _ in range(self.control_horizon):
                cumulative_state = dynamics.advance(cumulative_state, sample_action)


            pos_ee = dynamics.compute_fk(cumulative_state)
            vel_ee = dynamics.compute_vel_ee(cumulative_state)


            distance = np.linalg.norm(goal - pos_ee)
            velocity_magnitude = np.linalg.norm(vel_ee)


            if distance < 0.05:
                total_cost =  velocity_magnitude
            else:
                 total_cost = distance + 0.01 * velocity_magnitude

            if total_cost < min_cost:
                min_cost = total_cost
                best_action = sample_action

        return best_action

# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math

torch.manual_seed(0)

np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()
action = np.zeros((arm.dynamics.get_action_dim(), 1))
action[0] = 0

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    action = controller.compute_action(arm.dynamics, state, goal, action)
    arm.set_action(action)

At timestep 0: Distance to goal: 3.3970575502926055, Velocity of end effector: 5.539171676143414e-18
At timestep 5: Distance to goal: 3.111133494632821, Velocity of end effector: 7.8159747591600865
At timestep 10: Distance to goal: 2.62792054631962, Velocity of end effector: 4.8826095817651245
At timestep 15: Distance to goal: 2.1735383468302056, Velocity of end effector: 1.2107067810039385
At timestep 20: Distance to goal: 2.1464747136619913, Velocity of end effector: 5.592084823892467
At timestep 25: Distance to goal: 2.0684826921898956, Velocity of end effector: 5.443062795801731
At timestep 30: Distance to goal: 1.927671743448132, Velocity of end effector: 3.7743195313694735
At timestep 35: Distance to goal: 1.796381461139741, Velocity of end effector: 2.799623029483955
At timestep 40: Distance to goal: 1.6220804615980613, Velocity of end effector: 4.478533234789687
At timestep 45: Distance to goal: 1.3585644878609804, Velocity of end effector: 7.70625067020996
At timestep 50: Dist

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.42009822 -0.90747864], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.6498049  -0.76010104], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.62540735 -1.60533028], Final velocity: [0.30221636]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.14487976 -1.26090388], Final velocity: [0.08351861]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.26237575 -2.33711346], Final velocity: [0.07745141]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.09232452 -1.40159862], Final velocity: [0.16704938]
score: 1.0/1.0
       
---------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
	def __init__(self, num_links, time_step):
		super().__init__()
		self.num_links = num_links
		self.time_step = time_step

	def forward(self, x):
		qddot = self.compute_qddot(x)
		state = x[:, :2*self.num_links]
		next_state = self.compute_next_state(state, qddot)
		return next_state

	def compute_next_state(self, state, qddot):
		# Your code goes here

		q = state[:, :self.num_links]
		q_dot = state[:, self.num_links:2*self.num_links]

		q_next = q + self.time_step * q_dot + 0.5 * self.time_step ** 2 * qddot
		q_dot_next = q_dot + self.time_step * qddot

		next_state = torch.cat((q_next, q_dot_next), dim=1)
		return next_state

	def compute_qddot(self, x):
		pass

class Model2Link(Model):
  def __init__(self, time_step):
    super().__init__(2, time_step)
    # Your code goes here
    torch.manual_seed(0)

    self.fc1 = nn.Linear(6, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 128)
    self.fc4 = nn.Linear(128, 64)
    self.fc5 = nn.Linear(64, 32)
    self.fc6 = nn.Linear(32, 16)
    self.fc7 = nn.Linear(16, 2)



  def compute_qddot(self, x):
    # Your code goes here
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = F.relu(self.fc6(x))
    x = self.fc7(x)
    return x


### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

def collect_data(arm):

  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  np.random.seed(0)
  random.seed(42)
  X = np.zeros((0, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((0, arm.dynamics.get_state_dim()))
  state = np.zeros((arm.dynamics.get_state_dim(), 1))  # position and velocity
  state[0] = -math.pi / 2.0
  # We run the simulator for 5 seconds with a time step of 0.01 second,
  # so there are 500 steps in total
  sampling_points = 10

  # Controller
  controller = MPC()
  for k in range(sampling_points):
    print("traget number:",k)
    # Resetting the arm will set its state so that it is in the vertical position,
    # and set the action to be zeros
    arm.reset()
    action = np.zeros((arm.dynamics.get_action_dim(), 1))
    action[0] = 0

    # Choose the goal position you would like to see the performance of your controller
    goal = np.zeros((2, 1))
    goal[0, 0] = random.uniform(-2,2)
    goal[1, 0] = random.uniform(-2,2)
    arm.goal = goal

    dt = 0.01
    time_limit = 2.5
    num_steps = round(time_limit/dt)

    # Control loop
    for s in range(num_steps):

      t = time.time()
      arr1 = state
      arr2 = action


      result = np.concatenate((arr1, arr2), axis=0)

      transposed_result = result.transpose()

      X = np.append(X, transposed_result, axis=0)
      arm.advance()

      new_state = arm.get_state()

      transposed_new_state = new_state.transpose()

      Y = np.append(Y, transposed_new_state, axis=0)

      if s % controller.control_horizon==0:
        state = arm.get_state()

        # Measuring distance and velocity of end effector
        pos_ee = dynamics_teacher.compute_fk(state)
        dist = np.linalg.norm(goal-pos_ee)
        vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))

        action = controller.compute_action(arm.dynamics, state, goal, action)
        arm.set_action(action)

  return X, Y


In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))

traget number: 0
traget number: 1
traget number: 2
traget number: 3
traget number: 4
traget number: 5
traget number: 6
traget number: 7
traget number: 8
traget number: 9


### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time

np.random.seed(0)
torch.manual_seed(0)


class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]


def train_one_epoch(model, train_loader, optimizer, criterion):
  model.train()
	# ---
	# Your code goes here
	# ---
  total_loss = 0
  for X, Y in train_loader:
      #X, Y = X.to(device), Y.to(device)
      optimizer.zero_grad()
      output = model(X)
      loss = criterion(output, Y)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
  average_loss = total_loss / len(train_loader)
  return average_loss

def test(model, test_loader, criterion):
  model.eval()
	# --
	# Your code goes here
  # --
  total_loss = 0
  with torch.no_grad():
      for X, Y in test_loader:
          #X, Y = X.to(device), Y.to(device)
          output = model(X)
          loss = criterion(output, Y)
          total_loss += loss.item()
  average_loss = total_loss / len(test_loader)
  return average_loss


def train_forward_model():

	# --
	# Implement this function
  # --

  # Keep track of the checkpoint with the smallest test loss and save in model_path

  np.random.seed(0)
  torch.manual_seed(0)
  model_path = None
  max_test_loss = 1e4
  model = Model2Link(0.01)

  datafile = 'dataset/data.pkl'
  split = 0.2
  dataset = DynamicDataset(datafile)
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)

  epochs=200

  optimizer = torch.optim.Adam(model.parameters(), lr=0.0004) #0.0005
  criterion = nn.MSELoss()

  for epoch in range(1, 1 + epochs):
    # --
    # Your code goes here
    # --
    # ---
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    test_loss = test(model, test_loader, criterion)
    print(f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss}')

    # Save model if test loss improved
    if train_loss < max_test_loss:
        max_test_loss = train_loss
        model_folder_name = f'epoch_{epoch:04d}_loss_{train_loss:.8f}'
        model_save_path = os.path.join(model_dir, model_folder_name)
        if not os.path.exists(model_save_path):
            os.makedirs(model_save_path)
        model_file_path = os.path.join(model_save_path, 'dynamics.pth')
        torch.save(model.state_dict(), model_file_path)
        model_path = model_file_path

  print(model_path)
  return model_path

In [ ]:
model_path = train_forward_model()

Epoch: 1, Train Loss: 3.198172613978386, Test Loss: 3.396621197462082
Epoch: 2, Train Loss: 2.9612128362059593, Test Loss: 2.908816382288933
Epoch: 3, Train Loss: 2.1330257058143616, Test Loss: 2.0077633634209633
Epoch: 4, Train Loss: 1.6862443583086133, Test Loss: 1.7483233585953712
Epoch: 5, Train Loss: 1.500952960923314, Test Loss: 1.7898065149784088
Epoch: 6, Train Loss: 1.6241078628227115, Test Loss: 1.5891845151782036
Epoch: 7, Train Loss: 1.459092527627945, Test Loss: 1.5669974312186241
Epoch: 8, Train Loss: 1.508000260218978, Test Loss: 1.5538510642945766
Epoch: 9, Train Loss: 1.3766637668013573, Test Loss: 1.569199651479721
Epoch: 10, Train Loss: 1.513056268915534, Test Loss: 1.515360452234745
Epoch: 11, Train Loss: 1.3538877898827195, Test Loss: 1.454531192779541
Epoch: 12, Train Loss: 1.3480599941685796, Test Loss: 1.464259423315525
Epoch: 13, Train Loss: 1.3182780258357525, Test Loss: 1.5066136457026005
Epoch: 14, Train Loss: 1.3055922482162714, Test Loss: 1.418418101966381

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = None
        self.model = Model2Link(0.01)
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.to(device)
        self.model.eval()
        self.device = device
        # ---
        self.model_loaded = True

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # ---
            # Your code goes here
            # Use the loaded model to predict new state given the current state and action
            action_t = torch.tensor(action, dtype=torch.float32).view(1, -1).to(self.device)
            state_t = torch.tensor(state, dtype=torch.float32).view(1, -1).to(self.device)


            model_input = torch.cat((state_t, action_t), dim=1)

            with torch.no_grad():
                new_state = self.model(model_input).cpu().numpy().flatten()

            return new_state
            # ---
        else:
            return state

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = "models/2024-03-31_20-07-46/epoch_0164_loss_0.55538117/dynamics.pth"
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 2.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


FileNotFoundError: [Errno 2] No such file or directory: 'models/2024-03-31_20-07-46/epoch_0164_loss_0.55538117/dynamics.pth'

## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
model_path = 'models/2024-04-06_23-09-33/epoch_0184_loss_0.53020567/dynamics.pth'
gui= False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.68373842 -0.85240533], Final position: [ 0.64119895 -0.87762988], Final velocity: [0.38844299]
score: 0.5/0.5
Test  2
Success! :)
 Goal: [ 0.16797797 -1.18338791], Final position: [ 0.18512639 -1.20146889], Final velocity: [0.44903084]
score: 0.5/0.5
Test  3
Success! :)
 Goal: [ 0.37827852 -0.76670386], Final position: [ 0.43037694 -0.78273868], Final velocity: [0.14890197]
score: 0.5/0.5
Test  4
Success! :)
 Goal: [ 0.83095725 -0.29394504], Final position: [ 0.78807662 -0.31846567], Final velocity: [0.03446951]
score: 0.5/0.5
Test  5
Success! :)
 Goal: [-0.67347864 -1.75625573], Final position: [-0.66194483 -1.81394696], Final velocity: [0.04606983]
score: 0.5/0.5
Test  6
Success! :)
 Goal: [ 0.14089751 -1.54787812], Final position: [ 0.15586033 -1.549725  ], Final velocity: [0.27300378]
score: 0.5/0.5
Test  7
Success! :)
 Goal: [ 1.09857465 -0.26

Time Limit
*   MPC evaluation (Part 1): 20 minutes
*   Data Collection with MPC (Part 2): 1 hour
*   Training the forward dynamics model (Part 3): 30 minutes


Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy. Using Ray is optional